In [6]:
try:
    import mediapipe
    import cv2
    import pyautogui
    import math
    import traceback
    import mouse
    capture = cv2.VideoCapture(0)
    fw = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    fh = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    sw, sh= pyautogui.size()
    
    def show(i,t):
        coor = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh)
        cv2.putText(frame,t, coor, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        if coor is not None:
            return list(coor)
        
    def scale(coord):
        x1,y1=coord[0]*(sw/fw)-(sw/2),coord[1]*(sh/fh)-(sh/2)
        x2,y2=(x1*1.8)+(sw/2),(y1*1.8)+(sh/2)
        return [round(x2),round(y2)]
    
    def ratio(a,b,c):
        mean=(math.dist(a,b)+math.dist(a,c))/2
        r= mean/math.dist(b,c)
        if r>3:
            return [1,r]
        else:
            return [0,r]
        
    
    with mediapipe.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=1) as hands:
        clicked=0
        cin=[0,0]
        while capture.isOpened():
            ret, frame = capture.read()
            if ret == False:
                continue
            frame = cv2.flip(frame, 1)
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks != None:
                index = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                thumb=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.THUMB_TIP]
                little=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.PINKY_TIP]
                cindex=show(index,"1")
                cthumb=show(thumb,"2")
                clittle=show(little,"3")
                if cthumb is not None and clittle is not None and cindex is not None:
                    a=scale(cindex)
                    b=scale(cthumb)
                    c=scale(clittle)
                    if math.dist(a,cin)>10:
                        mouse.move(a[0], a[1])
                        cin=a
                    if ratio(a,b,c)[0]==1 and clicked==0:
                        mouse.click("left")  
                        clicked=1
                    else:
                        clicked=0
                    cv2.putText(frame,"2 to 3 -> "+str(round(math.dist(b,c))),(20, 140),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 5)
                    cv2.putText(frame,"1 to 3 -> "+str(round(math.dist(a,c))),(20, 210),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 5)
                    cv2.putText(frame,"1 to 2 -> "+str(round(math.dist(a,b))),(20, 280),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 5)
                    cv2.putText(frame,"Ratio -> "+str(round(ratio(a,b,c)[1],2)),(20, 350),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 5)
                    cv2.putText(frame,str(a),(20, 70),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 5)
            cv2.imshow('Mouse Controller', frame)
            if cv2.waitKey(1) == 27: #escape key
                break
    cv2.destroyAllWindows()
    capture.release()

except:
    cv2.destroyAllWindows()
    capture.release()
    traceback.print_exc()